In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2023-08-20 00:02:21.206301: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


데이터 셋 로드 (TFDS에서 가져온 EMNIST)

In [2]:
(etfds_train, etfds_test), etfds_info = tfds.load("emnist", split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
print(etfds_info)
print(etfds_train)

def normalize_img_ascii_lable(image, label):
    ascii_Lb = label
    if ascii_Lb >= 10:
        ascii_Lb += 7
    if ascii_Lb >= 36:
        ascii_Lb += 6
    ascii_Lb += 16
    return tf.cast(image, tf.float32) / 255., ascii_Lb

etfds_train = etfds_train.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_train = etfds_train.cache()
etfds_train = etfds_train.shuffle(etfds_info.splits['train'].num_examples)
etfds_train = etfds_train.batch(128)
etfds_train = etfds_train.prefetch(tf.data.experimental.AUTOTUNE)
print(etfds_train)

etfds_test = etfds_test.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_test = etfds_test.batch(128)
etfds_test = etfds_test.cache()
etfds_test = etfds_test.prefetch(tf.data.experimental.AUTOTUNE)

tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/byclass/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST ByClass
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_path='/Users/varzero/tensorflow_datasets/emnist/byclass/3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=349.16 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=62),
    }),
    supe

직접 만든 데이터 셋 로드

In [3]:
import pathlib

data_dir = pathlib.Path('../dataset')

trainFG = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(28,28),
    batch_size=16,
    label_mode='int',
    color_mode="grayscale"
)



testFG = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(28,28),
    batch_size=16,
    label_mode='int',
    color_mode="grayscale"
)

def normalize_myimg_ascii_lable(image, label):
    return tf.cast(image, tf.float32) / 255., tf.cast(int(label), dtype=tf.int64)

trainFG = trainFG.map(normalize_myimg_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
trainFG = trainFG.cache()
trainFG = trainFG.prefetch(tf.data.experimental.AUTOTUNE)
print(trainFG)

testFG = testFG.map(normalize_myimg_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
testFG = testFG.cache()
testFG = testFG.prefetch(tf.data.experimental.AUTOTUNE)


Found 1520 files belonging to 95 classes.
Using 1216 files for training.
Found 1520 files belonging to 95 classes.
Using 304 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


모델 세팅

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(95, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 95)                48735     
                                                                 
Total params: 450655 (1.72 MB)
Trainable params: 450655 (1.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


훈련시키기

In [5]:
model.fit(etfds_train, epochs=5)
model.fit(trainFG, epochs=10)

Epoch 1/5
5453/5453 [==============================] - 82s 8ms/step - loss: 0.6849 - accuracy: 0.7860
Epoch 2/5
5453/5453 [==============================] - 44s 8ms/step - loss: 0.5139 - accuracy: 0.8259
Epoch 3/5
5453/5453 [==============================] - 43s 8ms/step - loss: 0.4847 - accuracy: 0.8329
Epoch 4/5
5453/5453 [==============================] - 46s 8ms/step - loss: 0.4682 - accuracy: 0.8373
Epoch 5/5
5453/5453 [==============================] - 45s 8ms/step - loss: 0.4576 - accuracy: 0.8399
Epoch 1/10
76/76 [==============================] - 1s 7ms/step - loss: 5.8002 - accuracy: 0.0567
Epoch 2/10
76/76 [==============================] - 0s 6ms/step - loss: 2.3953 - accuracy: 0.4465
Epoch 3/10
76/76 [==============================] - 0s 5ms/step - loss: 1.2005 - accuracy: 0.7434
Epoch 4/10
76/76 [==============================] - 0s 6ms/step - loss: 0.6604 - accuracy: 0.8725
Epoch 5/10
76/76 [==============================] - 0s 5ms/step - loss: 0.3842 - accuracy: 0.9408


모델 저장하기

In [6]:
model.save('spemnist.keras')

확인하기

In [7]:
model.evaluate(etfds_test, verbose=2)
model.evaluate(testFG, verbose=2)

909/909 - 9s - loss: 5.1407 - accuracy: 0.2665 - 9s/epoch - 10ms/step
19/19 - 0s - loss: 2.5612 - accuracy: 0.4803 - 152ms/epoch - 8ms/step


[2.5611512660980225, 0.4802631437778473]